In [ ]:
from functions import *
# full_lit_df = generate_literature_csv_from_pdf("data/weather.pdf", "Author", "Title", "Publication Year", output_path="full_lit.csv")
df = autosearch_missing_abstracts("full_lit.csv", "trash.csv")
column arguments

In [ ]:
def extract_focal_claims(pdf_path: str, max_claims = 5) -> list:
    loader = PyPDFLoader(pdf_path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=200)
    docs = loader.load_and_split(text_splitter=text_splitter)
    claims = []
    for i, doc in enumerate(docs):
        if i >= max_claims:
            is there a cleverer way
            break
        prompt = f"Extract the novel claim from the following research paper snippet. Only give the claim that is novel and unique to the publication. Ignore claims from previous references mentioned in the paper. Simply reproduce the novel claim within one sentence (Example responses: 'A low sodium diet reduces the risk of heart disease.', 'The attention mechanism was not pioneered in transformer architectures.'). If there is no claim, return an empty string. Paper snippet: <<<{doc.page_content}>>>"
        system_prompt = "You are an AI model that extracts the central claim from research papers. You state the claim in a single sentence as if you were the original author."
        response = perform_chat_completion(prompt= prompt, system_message=system_prompt, temperature=0)
        claims.append(response)
    return claims
claims = extract_focal_claims("data/manuscript.pdf")
for claim in claims:
    print(claim)
    print()

In [ ]:
# from functions import *
# presented_claim = "Humor is more important for dating success than physical attractiveness." 
# zotero_file = "data/humorXdating_literature.csv"
# data_formatted = format_texts_from_csv(zotero_file, "Abstract Note", "Author", "Title", "Publication Year")
# print(data_formatted[["file_content", "reference"]].head(3))
# vectorstore_folder_name = generate_vectorstore(data_formatted, zotero_file, max_doc_size = 4000)
# relevant_docs = get_relevant_docs(presented_claim, vectorstore_folder_name)
# responses, summary_response = generate_adversarial_texts(presented_claim, relevant_docs)